# Test Access to Earth Engine

Run the code blocks below to test if the notebook server is authorized to communicate with the Earth Engine backend servers. Note that the code block may minimized, but you can expand them before running them.

First, check if the IPython Widgets library is available on the server.

In [ ]:
# Code to check the IPython Widgets library.
try:
  import ipywidgets
  print('The IPython Widgets library (version {0}) is available on this server.'.format(
      ipywidgets.__version__
    ))
except ImportError:
  print('The IPython Widgets library is not available on this server.\n'
        'Please see https://github.com/jupyter-widgets/ipywidgets '
        'for information on installing the library.')
  raise

Next, check if the Earth Engine API is available on the server.

In [ ]:
# Code to check the Earth Engine API library.
try:
  import ee
  print('The Earth Engine Python API (version {0}) is available on this server.'.format(
      ee.__version__
    ))
except ImportError:
  print('The Earth Engine Python API library is not available on this server.\n'
        'Please see https://developers.google.com/earth-engine/python_install '
        'for information on installing the library.')
  raise

Finally, check if the notebook server is authorized to access the Earth Engine backend servers.

In [ ]:
# Code to check if authorized to access Earth Engine.
import cStringIO
import os
import urllib

def isAuthorized():
  try:
    ee.Initialize()
    return True
  except:
    return False

form_item_layout = ipywidgets.Layout(width="100%", align_items='center')
  
if isAuthorized():
  
  def revoke_credentials(sender):
    credentials = ee.oauth.get_credentials_path()
    if os.path.exists(credentials):
      os.remove(credentials)
    print('Credentials have been revoked.')
  
  # Define widgets that may be displayed.
  auth_status_button = ipywidgets.Button(
    layout=form_item_layout,
    disabled = True,
    description = 'The server is authorized to access Earth Engine',
    button_style = 'success',
    icon = 'check'
  )
  
  instructions = ipywidgets.Button(
    layout=form_item_layout,
    description = 'Click here to revoke authorization',
    button_style = 'danger',
    disabled = False,
  )
  instructions.on_click(revoke_credentials)

else:
  
  def save_credentials(sender):
    try:
      token = ee.oauth.request_token(get_auth_textbox.value.strip())
    except Exception as e:
      print(e)
      return
    ee.oauth.write_token(token)
    get_auth_textbox.value = ''  # Clear the textbox.
    print('Successfully saved authorization token.')

  # Define widgets that may be displayed.
  get_auth_textbox = ipywidgets.Text(
    placeholder='Paste authorization code here',
    description='Authentication Code:'
  )
  get_auth_textbox.on_submit(save_credentials)

  auth_status_button = ipywidgets.Button(
    layout=form_item_layout,
    button_style = 'danger',
    description = 'The server is not authorized to access Earth Engine',
    disabled = True
  )
  
  instructions = ipywidgets.VBox(
    [
      ipywidgets.HTML(
        'Click on the link below to start the authentication and authorization process. '
        'Once you have received an authorization code, paste it in the box below and press return.'
      ),
      ipywidgets.HTML(
        '<a href="{url}" target="auth">Open Authentication Tab</a><br/>'.format(
          url=ee.oauth.get_authorization_url()
        )
      ),
      get_auth_textbox
    ],
    layout=form_item_layout
  )

# Display the form.
form = ipywidgets.VBox([
  auth_status_button,
  instructions
])
form

Once the server is authorized, you can retrieve data from Earth Engine and use it in the notebook.

In [ ]:
# Code to display an Earth Engine generated image.
from IPython.display import Image

url = ee.Image("CGIAR/SRTM90_V4").getThumbUrl({'min':0, 'max':3000})
Image(url=url)